[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Fine-Tune YOLO for Quality Inspection**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

This notebook implements a complete pipeline for training a computer vision model and using it to inspect bottle quality on a manufacturing assembly line. It covers data preparation, model training, and a video processing inference loop.

#### **Key Features**
*   **Data Preparation**: Includes utilities to extract frames from videos and convert COCO-JSON annotations to YOLO segmentation format for training.
*   **Model Training**: Fine-tunes a **YOLOv11 segmentation model** (`yolo11m-seg.pt`) on a custom bottle dataset.
*   **Interactive ROI Selector**: Provides a UI tool (`get_polygon_points`) to interactively draw and define a "Quality Inspection Zone" polygon on a video frame.
*   **Quality Inspection Logic**:
    *   **Component Detection**: Identifies bottles, caps, and labels.
    *   **Spatial Verification**: Uses intersection-over-union calculations to verify if a bottle has both a cap and a label properly attached.
    *   **Zone Filtering**: Only inspects bottles that pass through the defined polygon zone.
*   **Visual Feedback**:
    *   **Dynamic Annotations**: Draws color-coded bounding boxes (Green/Bottle, Blue/Cap, Orange/Label) with "PASS" status indicators.
    *   **Zone Visualization**: Displays the inspection zone as a red dotted line.
    *   **Counters**: Tracks and displays a running count of passed bottles on-screen.

#### **Workflow**
1.  **Setup**: Import libraries and install custom YOLO finetuning utilities.
2.  **Train**: Prepare dataset and fine-tune the YOLO model (300 epochs).
3.  **Configure**: Interactively select the region of interest (ROI) from the input video.
4.  **Inference**: Run the `BottleQualityInspector` class to process the video, tracking bottles and validating their assembly quality in real-time.

## Import Libraries
Import necessary libraries for computer vision, visualization, and deep learning.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import torch
from typing import List, Tuple, Dict
from pathlib import Path
%matplotlib inline

## Install Utilities
Clone the YOLO finetuning utilities repository.

In [ ]:
# uncommet the following line if you have not cloned the repository
# !git clone https://github.com/Labellerr/yolo_finetune_utils.git

## Extract Frames (Optional)
Extract frames from video for dataset creation. (Currently commented out)

In [ ]:
from yolo_finetune_utils.frame_extractor import extract_random_frames

extract_random_frames(
        paths=[r"videos\manufacturing_video_data"],
        total_images=150,
        out_dir="manufacturing_dataset_frames",
        jpg_quality=100,
        seed=42
    )

## Convert Annotations
Convert COCO JSON annotations to YOLO format for segmentation.

In [ ]:
from yolo_finetune_utils.coco_yolo_converter.seg_converter import coco_to_yolo_converter

ANNOTATION_JSON = "annotations.json"
IMAGE_DIR = "manufacturing_dataset_frames"


coco_to_yolo_converter(
        json_path=ANNOTATION_JSON,
        images_dir=IMAGE_DIR,
        output_dir="yolo_dataset",
        use_split=True,
        train_ratio=0.7,
        val_ratio=0.1,
        test_ratio=0.1,
        shuffle=True,
        verbose=False
    )

## Train YOLO Model
Fine-tune the YOLOv11 segmentation model on the custom dataset.

In [ ]:
from ultralytics import YOLO
# Load a model
model = YOLO("yolo11m-seg.pt")

# Train the model
results = model.train(
    data=r"yolo_dataset\data.yaml",    # Path to your dataset YAML file
    epochs=300,                        # Number of training epochs
    imgsz=640,                         # Image size
    batch=-1,                          # Batch size
    device=0,                          # GPU device (0 for first GPU, 'cpu' for CPU)
    workers=4                          # Number of dataloader workers
)

In [ ]:
TEST_IMG = r"yolo_dataset\images\test\bottle manufacturing_frame_000225_t7.50s_000036.jpg"
result = model.predict(TEST_IMG, show_labels=False)

plt.figure(figsize=(12, 8))
plt.imshow(result[0].plot(conf=False, labels=False)[:, :, ::-1]
)
plt.axis('off')
plt.show()

In [ ]:
results = model.predict(video_path, save=True, conf=0.5, show_labels=False, stream=True)
for result in results:
    pass

## Helper: Extract and Display Frame
Define a function to extract and display a specific frame from a video.

In [ ]:

def extract_nth_frame(video_path, frame_number):
    """
    Extract the nth frame from a video and display it using matplotlib.
    
    Parameters:
    -----------
    video_path : str
        Path to the video file
    frame_number : int
        The frame number to extract (0-indexed)
    
    Returns:
    --------
    frame : numpy.ndarray
        The extracted frame in RGB format, or None if extraction fails
    """
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return None
    
    # Get total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Check if the requested frame number is valid
    if frame_number < 0 or frame_number >= total_frames:
        print(f"Error: Frame number {frame_number} is out of range (0-{total_frames-1})")
        cap.release()
        return None
    
    # Set the frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    
    # Read the frame
    ret, frame = cap.read()
    
    if not ret:
        print(f"Error: Could not read frame {frame_number}")
        cap.release()
        return None
    
    # Release the video capture object
    cap.release()
    
    # Convert BGR (OpenCV format) to RGB (matplotlib format)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Display the frame using matplotlib
    plt.figure(figsize=(12, 8))
    plt.imshow(frame_rgb)
    plt.title(f'Frame {frame_number}')
    plt.axis('off')
    plt.tight_layout()
    plt.show()


## Visualize Reference Frame
Extract and display specific frames to check the video content.

In [ ]:
VIDEO_PATH = r"manufacturing_video_data\bottle manufacturing.mp4"

frame_no = [0, 30, 50, 100]

for frame in frame_no:
    extract_nth_frame(VIDEO_PATH, frame)

## Helper: Interactive ROI Selector
Define a function to interactively draw a polygon ROI on a video frame.

In [ ]:
def get_polygon_points(video_path, frame_no=0):
    """
    Draw a polygon on a video frame using OpenCV interactive UI.
    
    Controls:
    - Left-click: Add point to polygon
    - Right-click: Finish polygon (minimum 3 points)
    - Press 'r': Reset and start over
    - Press 'c': Complete polygon (same as right-click)
    - Press 'Esc': Cancel and exit
    
    Parameters:
    -----------
    video_path : str
        Path to the video file
    frame_no : int
        Frame number to extract (default: 0)
    
    Returns:
    --------
    list : List of polygon points [(x1, y1), (x2, y2), ...]
           Returns None if drawing failed or cancelled
    
    Example:
    --------
    >>> polygon = get_polygon_points("video.mp4", frame_no=100)
    >>> print(polygon)
    [(100, 200), (300, 200), (300, 400), (100, 400)]
    """
    
    # Extract frame
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Error: Could not open video {video_path}")
        return None
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print(f"❌ Error: Could not read frame {frame_no}")
        return None
    
    # State variables
    points = []
    drawing_complete = False
    frame_copy = frame.copy()
    
    # Mouse callback function
    def mouse_callback(event, x, y, flags, param):
        nonlocal points, drawing_complete
        
        # Left click - add point
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            print(f"✓ Point {len(points)}: ({x}, {y})")
        
        # Right click - finish polygon
        elif event == cv2.EVENT_RBUTTONDOWN:
            if len(points) >= 3:
                drawing_complete = True
                print(f"✓ Polygon complete: {len(points)} points")
            else:
                print(f"⚠ Need at least 3 points (currently have {len(points)})")
    
    # Create window
    window_name = "Draw Polygon Perimeter"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.setMouseCallback(window_name, mouse_callback)
    
    # Print instructions
    print("\n" + "="*70)
    print("POLYGON DRAWING - INTERACTIVE UI")
    print("="*70)
    print("  Left-click:  Add point to polygon")
    print("  Right-click: Finish polygon (minimum 3 points)")
    print("  Press 'r':   Reset and start over")
    print("  Press 'c':   Complete polygon")
    print("  Press 'Esc': Cancel and exit")
    print("="*70 + "\n")
    
    while True:
        # Create display frame
        display_frame = frame_copy.copy()
        
        # Draw all points
        for i, point in enumerate(points):
            # Draw point
            cv2.circle(display_frame, point, 6, (0, 255, 0), -1)
            cv2.circle(display_frame, point, 8, (255, 255, 255), 2)
            
            # Add point number
            cv2.putText(display_frame, str(i+1), 
                       (point[0] + 12, point[1] - 12),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(display_frame, str(i+1), 
                       (point[0] + 12, point[1] - 12),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)
        
        # Draw lines between consecutive points
        if len(points) > 1:
            for i in range(len(points) - 1):
                cv2.line(display_frame, points[i], points[i+1], (0, 255, 0), 2)
        
        # Draw closing line and fill if polygon is complete or has 3+ points
        if len(points) >= 3:
            # Draw closing line
            cv2.line(display_frame, points[-1], points[0], (0, 255, 0), 2)
            
            # Fill polygon with semi-transparent overlay
            overlay = display_frame.copy()
            pts = np.array(points, dtype=np.int32)
            cv2.fillPoly(overlay, [pts], (0, 255, 0))
            cv2.addWeighted(overlay, 0.3, display_frame, 0.7, 0, display_frame)
        
        # Add status text at the top
        status_text = f"Points: {len(points)}"
        if drawing_complete:
            status_text = f"COMPLETE - {len(points)} points (Press any key to exit)"
            cv2.rectangle(display_frame, (5, 5), (750, 45), (0, 200, 0), -1)
        else:
            cv2.rectangle(display_frame, (5, 5), (600, 45), (0, 0, 0), -1)
        
        cv2.putText(display_frame, status_text, (15, 32),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        
        # Show frame
        cv2.imshow(window_name, display_frame)
        
        # Handle keyboard input
        key = cv2.waitKey(1) & 0xFF
        
        # Esc - cancel
        if key == 27:
            print("❌ Cancelled by user")
            cv2.destroyAllWindows()
            return None
        
        # 'r' - reset
        elif key == ord('r') or key == ord('R'):
            points = []
            drawing_complete = False
            print("🔄 Reset - draw a new polygon")
        
        # 'c' - complete
        elif key == ord('c') or key == ord('C'):
            if len(points) >= 3:
                drawing_complete = True
                print(f"✓ Polygon complete: {len(points)} points")
            else:
                print(f"⚠ Need at least 3 points (currently have {len(points)})")
        
        # Exit if drawing is complete
        if drawing_complete:
            cv2.waitKey(1500)  # Show final result for 1.5 seconds
            break
    
    cv2.destroyAllWindows()
    
    if len(points) < 3:
        print(f"⚠ No valid polygon drawn (need at least 3 points)")
        return None
    
    print(f"\n✅ Polygon saved with {len(points)} vertices")
    print(f"Coordinates: {points}\n")
    return points


## Define Region of Interest
Launch the interactive tool to define the polygon ROI on a selected frame.

In [ ]:
region_of_interest = get_polygon_points(VIDEO_PATH, 50)

## Helper: Annotate ROI
Define a function to verify the selected ROI by overlaying it on the frame.

In [ ]:
def annotate_frame_with_polygon(video_path, frame_number, polygon_coords):
    """
    Annotate a specific video frame with a polygon region.
    
    Parameters:
    -----------
    video_path : str
        Path to the video file.
    frame_number : int
        The frame number to extract and annotate.
    polygon_coords : list
        List of (x, y) tuples defining the polygon vertices.
        Example: [(2, 2150), (1262, 1348), (3838, 1402), (3838, 2155), (15, 2158)]
    """
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    # Set frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print(f"Error: Could not read frame {frame_number}")
        return
    # Create a copy for drawing
    annotated_frame = frame.copy()
    
    # Convert polygon coordinates to numpy array
    pts = np.array(polygon_coords, np.int32)
    pts = pts.reshape((-1, 1, 2))
    
    # Draw polygon: Magenta color, thickness 5
    cv2.polylines(annotated_frame, [pts], isClosed=True, color=(255, 0, 255), thickness=5)
    
    # Optional: Fill polygon with semi-transparent overlay
    overlay = annotated_frame.copy()
    cv2.fillPoly(overlay, [pts], (255, 0, 255))
    cv2.addWeighted(overlay, 0.2, annotated_frame, 0.8, 0, annotated_frame)
    # Convert to RGB for matplotlib display
    frame_rgb = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(frame_rgb)
    plt.title(f"Frame {frame_number} with ROI")
    plt.axis('off')
    plt.show()

## Verify ROI Annotation
Display the frame with the defined polygon overlay to verify accuracy.

In [ ]:
annotate_frame_with_polygon(video_path, 50, region_of_interest)

## Quality Inspection class
Run the `BottleQualityInspector` class to process the video, tracking bottles and validating their assembly quality in real-time.

In [ ]:
class QualityInspector:
    """bottle quality inspection using YOLO segmentation."""
    
    def __init__(self, model_path: str, zone_polygon: List[Tuple[int, int]], 
                 conf_threshold: float = 0.5, tracker: str = 'bytetrack.yaml'):
        self.model = YOLO(model_path)
        self.zone_polygon = np.array(zone_polygon, dtype=np.int32)
        self.conf_threshold = conf_threshold
        self.tracker = tracker
        self.pass_count = 0
        self.tracked_bottles = {}  # {track_id: bool} passed status
    
    def _is_box_in_zone(self, box: np.ndarray) -> bool:
        """Check if box center-bottom is inside zone polygon."""
        x1, y1, x2, y2 = box
        center_bottom = ((x1 + x2) / 2, y2)
        return cv2.pointPolygonTest(self.zone_polygon, center_bottom, False) >= 0
    
    def _calc_overlap(self, bottle_box: np.ndarray, comp_box: np.ndarray) -> float:
        """Calculate overlap percentage of component with bottle."""
        b_x1, b_y1, b_x2, b_y2 = bottle_box
        c_x1, c_y1, c_x2, c_y2 = comp_box
        x_left, y_top = max(b_x1, c_x1), max(b_y1, c_y1)
        x_right, y_bottom = min(b_x2, c_x2), min(b_y2, c_y2)
        if x_right < x_left or y_bottom < y_top: return 0.0
        intersection = (x_right - x_left) * (y_bottom - y_top)
        comp_area = (c_x2 - c_x1) * (c_y2 - c_y1)
        return (intersection / comp_area * 100) if comp_area > 0 else 0.0
    
    def _verify_quality(self, bottle_box, caps, labels, threshold=10.0) -> bool:
        """Check if bottle has cap AND label with sufficient overlap."""
        has_cap = any(self._calc_overlap(bottle_box, c) > threshold for c in caps)
        has_label = any(self._calc_overlap(bottle_box, l) > threshold for l in labels)
        return has_cap and has_label
    
    def _separate_by_class(self, results) -> Tuple[List, List, List]:
        """Separate detections: class 0=label, 1=bottle, 2=cap (from data.yaml)"""
        bottles, caps, labels = [], [], []
        if results[0].boxes is None or len(results[0].boxes) == 0:
            return bottles, caps, labels
        
        boxes = results[0].boxes.xyxy.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        confs = results[0].boxes.conf.cpu().numpy()
        track_ids = results[0].boxes.id.cpu().numpy() if results[0].boxes.id is not None else np.arange(len(boxes))
        
        for box, cls, conf, tid in zip(boxes, classes, confs, track_ids):
            if conf < self.conf_threshold: continue
            cls, tid = int(cls), int(tid)
            detection = (box, tid, conf)
            if cls == 0: labels.append(detection)  # bottle label
            elif cls == 1 and self._is_box_in_zone(box): bottles.append(detection)  # bottle
            elif cls == 2: caps.append(detection)  # bottle cap
        return bottles, caps, labels
    
    def _draw_dotted_line(self, frame, pt1, pt2, color, thickness=4, gap=15):
        """Draw dotted line between two points."""
        import math
        dist = math.sqrt((pt2[0]-pt1[0])**2 + (pt2[1]-pt1[1])**2)
        pts = [(int(pt1[0]*(1-i/dist)+pt2[0]*i/dist), int(pt1[1]*(1-i/dist)+pt2[1]*i/dist)) 
               for i in range(0, int(dist), gap)]
        for i in range(0, len(pts)-1, 2):
            if i+1 < len(pts): cv2.line(frame, pts[i], pts[i+1], color, thickness)
    
    def _annotate_frame(self, frame, bottles, caps, labels, quality_results) -> np.ndarray:
        """Draw annotations on frame."""
        annotated = frame.copy()
        
        # Red dotted zone polygon
        pts = self.zone_polygon.reshape(-1, 2)
        for i in range(len(pts)):
            self._draw_dotted_line(annotated, tuple(pts[i]), tuple(pts[(i+1)%len(pts)]), (0,0,255), 4, 15)
        
        # Zone label "Quality Inspection"
        zone_label = "Quality Inspection"
        label_x, label_y = pts[0][0], pts[0][1] - 10  # Above top-left corner
        cv2.putText(annotated, zone_label, (label_x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2, cv2.LINE_AA)
        
        # Match caps/labels to bottles
        cap_boxes, label_boxes = [c[0] for c in caps], [l[0] for l in labels]
        confirmed_caps, confirmed_labels = set(), set()
        for bx, _, _ in bottles:
            for i, cb in enumerate(cap_boxes):
                if self._calc_overlap(bx, cb) > 10: confirmed_caps.add(i)
            for i, lb in enumerate(label_boxes):
                if self._calc_overlap(bx, lb) > 10: confirmed_labels.add(i)
        
        # Draw bottles (GREEN)
        for box, tid, _ in bottles:
            x1, y1, x2, y2 = map(int, box)
            if quality_results.get(tid, False):
                overlay = annotated.copy()
                cv2.rectangle(overlay, (x1,y1), (x2,y2), (0,255,0), -1)
                cv2.addWeighted(overlay, 0.3, annotated, 0.7, 0, annotated)
            cv2.rectangle(annotated, (x1,y1), (x2,y2), (0,255,0), 3)
        
        # Draw caps (BLUE)
        for i, (box, _, _) in enumerate(caps):
            if i in confirmed_caps:
                x1, y1, x2, y2 = map(int, box)
                overlay = annotated.copy()
                cv2.rectangle(overlay, (x1,y1), (x2,y2), (255,100,0), -1)
                cv2.addWeighted(overlay, 0.4, annotated, 0.6, 0, annotated)
                cv2.rectangle(annotated, (x1,y1), (x2,y2), (255,100,0), 2)
                # Text with background
                text = "CAP: PASS"
                (text_w, text_h), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                cv2.rectangle(annotated, (x1+4, y1-text_h-8), (x1+text_w+12, y1-4), (255, 100, 0), -1)
                cv2.putText(annotated, text, (x1+8, y1-8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        
        # Draw labels (ORANGE)
        for i, (box, _, _) in enumerate(labels):
            if i in confirmed_labels:
                x1, y1, x2, y2 = map(int, box)
                overlay = annotated.copy()
                cv2.rectangle(overlay, (x1,y1), (x2,y2), (0,165,255), -1)
                cv2.addWeighted(overlay, 0.4, annotated, 0.6, 0, annotated)
                cv2.rectangle(annotated, (x1,y1), (x2,y2), (0,165,255), 2)
                # Text with background
                text = "LABEL: PASS"
                (text_w, text_h), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                cv2.rectangle(annotated, (x1+4, y1-text_h-8), (x1+text_w+12, y1-4), (0, 165, 255), -1)
                cv2.putText(annotated, text, (x1+8, y1-8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        
        return annotated
    
    def _draw_counter(self, frame) -> np.ndarray:
        """Draw pass counter in top-right corner."""
        h, w = frame.shape[:2]
        text = f"Bottle Passed: {self.pass_count}"
        font, scale, thick = cv2.FONT_HERSHEY_SIMPLEX, 1.2, 2
        (tw, th), _ = cv2.getTextSize(text, font, scale, thick)
        x1, y1 = w - tw - 40, 20
        overlay = frame.copy()
        cv2.rectangle(overlay, (x1-10, y1), (x1+tw+10, y1+th+20), (0,0,0), -1)
        cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
        cv2.putText(frame, text, (x1, y1+th+10), font, scale, (255,255,255), thick)
        return frame
    
    def process_frame(self, frame) -> Tuple[np.ndarray, Dict]:
        """Process single frame and return annotated frame + results."""
        results = self.model.track(frame, conf=self.conf_threshold, persist=True, tracker=self.tracker, verbose=False)
        bottles, caps, labels = self._separate_by_class(results)
        cap_boxes, label_boxes = [c[0] for c in caps], [l[0] for l in labels]
        
        quality_results = {}
        for box, tid, _ in bottles:
            if tid not in self.tracked_bottles:
                passed = self._verify_quality(box, cap_boxes, label_boxes)
                self.tracked_bottles[tid] = passed
                if passed: self.pass_count += 1
            quality_results[tid] = self.tracked_bottles[tid]
        
        annotated = self._annotate_frame(frame, bottles, caps, labels, quality_results)
        annotated = self._draw_counter(annotated)
        return annotated, {'bottles': len(bottles), 'caps': len(caps), 'labels': len(labels), 
                           'passed': sum(quality_results.values()), 'total_passed': self.pass_count}
    
    def process_video(self, video_path: str, output_path: str = None, show_progress: bool = True) -> Dict:
        """Process video file and optionally save output."""
        cap = cv2.VideoCapture(video_path)
        w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps, total = int(cap.get(cv2.CAP_PROP_FPS)), int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h)) if output_path else None
        if show_progress: print(f"Processing {video_path}: {w}x{h} @ {fps}fps, {total} frames")
        
        frame_num = 0
        while True:
            ret, frame = cap.read()
            if not ret: break
            annotated, _ = self.process_frame(frame)
            if writer: writer.write(annotated)
            frame_num += 1
            if show_progress and frame_num % 30 == 0:
                print(f"Progress: {frame_num/total*100:.1f}% ({frame_num}/{total})")
        
        cap.release()
        if writer: writer.release()
        if show_progress: print(f"\nComplete! {self.pass_count}/20 bottles passed")
        return {'total_frames': frame_num, 'pass_count': self.pass_count}

In [ ]:
# Define inspection zone polygon
zone = region_of_interest

# Initialize inspector
inspector = QualityInspector(
    model_path='runs/segment/train/weights/best.pt',
    zone_polygon=zone,
    conf_threshold=0.35,
    tracker='bytetrack.yaml'
)

# Process video
stats = inspector.process_video(
    video_path=VIDEO_PATH,
    output_path='quality_check_output.mp4'
)
print(f"Total bottles passed: {stats['pass_count']}")

---

## 👨‍💻 About Labellerr's Hands-On Learning in Computer Vision

Thank you for exploring this **Labellerr Hands-On Computer Vision Cookbook**! We hope this notebook helped you learn, prototype, and accelerate your vision projects.  
Labellerr provides ready-to-run Jupyter/Colab notebooks for the latest models and real-world use cases in computer vision, AI agents, and data annotation.

---
## 🧑‍🔬 Check Our Popular Youtube Videos

Whether you're a beginner or a practitioner, our hands-on training videos are perfect for learning custom model building, computer vision techniques, and applied AI:

- [How to Fine-Tune YOLO on Custom Dataset](https://www.youtube.com/watch?v=pBLWOe01QXU)  
  Step-by-step guide to fine-tuning YOLO for real-world use—environment setup, annotation, training, validation, and inference.
- [Build a Real-Time Intrusion Detection System with YOLO](https://www.youtube.com/watch?v=kwQeokYDVcE)  
  Create an AI-powered system to detect intruders in real time using YOLO and computer vision.
- [Finding Athlete Speed Using YOLO](https://www.youtube.com/watch?v=txW0CQe_pw0)  
  Estimate real-time speed of athletes for sports analytics.
- [Object Counting Using AI](https://www.youtube.com/watch?v=smsjBBQcIUQ)  
  Learn dataset curation, annotation, and training for robust object counting AI applications.
---

## 🎦 Popular Labellerr YouTube Videos

Level up your skills and see video walkthroughs of these tools and notebooks on the  
[Labellerr YouTube Channel](https://www.youtube.com/@Labellerr/videos):

- [How I Fixed My Biggest Annotation Nightmare with Labellerr](https://www.youtube.com/watch?v=hlcFdiuz_HI) – Solving complex annotation for ML engineers.
- [Explore Your Dataset with Labellerr's AI](https://www.youtube.com/watch?v=LdbRXYWVyN0) – Auto-tagging, object counting, image descriptions, and dataset exploration.
- [Boost AI Image Annotation 10X with Labellerr's CLIP Mode](https://www.youtube.com/watch?v=pY_o4EvYMz8) – Refine annotations with precision using CLIP mode.
- [Boost Data Annotation Accuracy and Efficiency with Active Learning](https://www.youtube.com/watch?v=lAYu-ewIhTE) – Speed up your annotation workflow using Active Learning.

> 👉 **Subscribe** for Labellerr's deep learning, annotation, and AI tutorials, or watch videos directly alongside notebooks!

---

## 🤝 Stay Connected

- **Website:** [https://www.labellerr.com/](https://www.labellerr.com/)
- **Blog:** [https://www.labellerr.com/blog/](https://www.labellerr.com/blog/)
- **GitHub:** [Labellerr/Hands-On-Learning-in-Computer-Vision](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
- **LinkedIn:** [Labellerr](https://in.linkedin.com/company/labellerr)
- **Twitter/X:** [@Labellerr1](https://x.com/Labellerr1)

*Happy learning and building with Labellerr!*
